![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-ocr-workshop/blob/master/tutorials/Certification_Trainings/6.1.SparkOcrStreamingPDF.ipynb)

## Spark OCR Streaming

## Blogposts and videos

- [Text Detection in Spark OCR](https://medium.com/spark-nlp/text-detection-in-spark-ocr-dcd8002bdc97)

- [Table Detection & Extraction in Spark OCR](https://medium.com/spark-nlp/table-detection-extraction-in-spark-ocr-50765c6cedc9)

- [Extract Tabular Data from PDF in Spark OCR](https://medium.com/spark-nlp/extract-tabular-data-from-pdf-in-spark-ocr-b02136bc0fcb)

- [Signature Detection in Spark OCR](https://medium.com/spark-nlp/signature-detection-in-spark-ocr-32f9e6f91e3c)

- [GPU image pre-processing in Spark OCR](https://medium.com/spark-nlp/gpu-image-pre-processing-in-spark-ocr-3-1-0-6fc27560a9bb)

- [How to Setup Spark OCR on UBUNTU - Video](https://www.youtube.com/watch?v=cmt4WIcL0nI)


**More examples here**

https://github.com/JohnSnowLabs/spark-ocr-workshop

In [ ]:
## Setup

In [ ]:
import os
from pyspark.ml import PipelineModel
import sparkocr
from pyspark.sql.functions import *
from sparkocr.transformers import *

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

spark = start_spark()
print("Spark OCR Version :", sparkocr.version())

In [2]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-ocr-workshop/master/jupyter/data/pdfs/noised.pdf

In [ ]:
# fill path to folder with PDF's here
dataset_path = "*.pdf"

In [ ]:
# read one file for infer schema
pdfs_df = spark.read.format("binaryFile").load(dataset_path).limit(1)
pdfs_df.show()

## Define OCR pipeline

In [ ]:
# Transform binary to image
pdf_to_image = PdfToImage()
pdf_to_image.setOutputCol("image")

# Run OCR for each region
ocr = ImageToText()
ocr.setInputCol("image")
ocr.setOutputCol("text")
ocr.setConfidenceThreshold(60)

# OCR pipeline
pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr
])

## Define streaming pipeline and start it
Note: each start erase previous results

In [ ]:
# count of files in one microbatch
maxFilesPerTrigger = 4

# read files as stream
pdf_stream_df = spark.readStream \
.format("binaryFile") \
.schema(pdfs_df.schema) \
.option("maxFilesPerTrigger", maxFilesPerTrigger) \
.load(dataset_path)

# process files using OCR pipeline
result = pipeline.transform(pdf_stream_df).withColumn("timestamp", current_timestamp())

# store results to memory table
query = result.writeStream \
 .format('memory') \
 .queryName('result') \
 .start()

In [ ]:
# get progress of streamig job
query.lastProgress

In [ ]:
# need to run for stop steraming job
query.stop()

## Show results from 'result' table

In [ ]:
# count of processed records (number of processed pages in results)
spark.table("result").count()

3

In [ ]:
# show results
spark.table("result").select("timestamp","pagenum", "path", "text").show(10)

+--------------------+-------+--------------------+--------------------+
|           timestamp|pagenum|                path|                text|
+--------------------+-------+--------------------+--------------------+
|2022-07-20 21:45:...|      0|file:/content/dat...|FOREWORD\n\nElect...|
|2022-07-20 21:45:...|      0|file:/content/dat...|C nca Document fo...|
|2022-07-20 21:56:...|      0|file:/content/dat...|6/13/22, 11:47 AM...|
+--------------------+-------+--------------------+--------------------+



## Run streaming job for storing results to disk

In [ ]:
query = result.select("text").writeStream \
 .format('text') \
 .option("path", "results/") \
 .option("checkpointLocation", "checkpointDir") \
 .start()

In [ ]:
# get progress of streamig job
query.lastProgress

In [ ]:
# need to run for stop steraming job
query.stop()

## Read results from disk

In [ ]:
results = spark.read.format("text").load("results/*.txt")
results.sample(.1).show(truncate=False)

+---------------------------------------------------------------------------------------------+
|value                                                                                        |
+---------------------------------------------------------------------------------------------+
|                                                                                             |
|                                                                                             |
|                                                                                             |
|                                                                                             |
|                                                                                             |
|                                                                                             |
|                                                                                             |
|                                       

## Clean results and checkpoint folders

In [ ]:
%%bash
rm -r -f results
rm -r -f checkpointDir